# Segmentation Example
> Train a U-Net for pixelwise segmentation of the prostate

In [1]:
import monai
import ignite

from prostate158.utils import load_config
from prostate158.train import SegmentationTrainer
from prostate158.report import ReportGenerator
from prostate158.viewer import ListViewer

### Installation Procedure of dependencies 

In [2]:
# # if you have a requirements.txt:
# !pip install --upgrade pip
# !pip install -r requirements.txt

# # Otherwise install core libs directly:
# !pip install monai["all"] ignite matplotlib pyyaml munch

# Finally
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# !pip install opencv-python
# !pip install ipywidgets

In [3]:
import torch

print(torch.cuda.is_available())  # For PyTorch

True


All parameters needed for training and evaluation are set in `anatomy.yaml` file. 

In [4]:
# config = load_config('anatomy.yaml') # change to 'anatomy.yaml' for anatomy segmentation
cfg = load_config("tumor.yaml")
monai.utils.set_determinism(seed=cfg.seed)
cfg.model.type = "rrunet3d"
# cfg.model.type = "unet"

In [5]:
from prostate158.model import get_model

model = get_model(cfg).to(cfg.device)  # Move the model to the selected device
print(model)

[get_model] model.type = 'rrunet3d'
RRUNet3D(
  (pool): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (unpool): Upsample(scale_factor=2.0, mode='nearest')
  (input_conv): Conv3d(3, 16, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
  (output_conv): Conv3d(16, 2, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
  (output_activation): Softmax(dim=1)
  (encoders): ModuleList(
    (0): ResidualBlock(
      (CONV): ConvBlock(
        (ops): ModuleList(
          (0): Conv3d(16, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
          (1): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
      )
      (RCNN): Sequential(
        (0): RecurrentBlock(
          (conv): Sequential(
            (0): Conv3d(16, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
            (1): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_s

Create supervised trainer for segmentation task

In [6]:
trainer = SegmentationTrainer(
    progress_bar=True,
    early_stopping=True,
    metrics=["MeanDice", "HausdorffDistance", "SurfaceDistance"],
    save_latest_metrics=True,
    config=cfg,
)

# # Load pre-trained weights from tumor.pt
# trainer.load_checkpoint("models/tumor.pt")

[get_model] model.type = 'rrunet3d'


Adding a learning rate scheduler for one-cylce policy. 

In [7]:
trainer.fit_one_cycle()

Let's train. This can take several hours. 

In [8]:
trainer.run()

`torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.


2025-08-10 16:23:14,290 - INFO - Epoch: 1/500, Iter: 1/119 -- train_loss: 1.3051 


[1/119]   1%|           [00:00<?]

2025-08-10 16:23:21,176 - INFO - Epoch: 1/500, Iter: 2/119 -- train_loss: 1.2622 
2025-08-10 16:23:21,586 - INFO - Epoch: 1/500, Iter: 3/119 -- train_loss: 1.3226 
2025-08-10 16:23:22,001 - INFO - Epoch: 1/500, Iter: 4/119 -- train_loss: 1.3328 
2025-08-10 16:23:22,356 - INFO - Epoch: 1/500, Iter: 5/119 -- train_loss: 1.3364 
2025-08-10 16:23:22,753 - INFO - Epoch: 1/500, Iter: 6/119 -- train_loss: 1.2986 
2025-08-10 16:23:23,159 - INFO - Epoch: 1/500, Iter: 7/119 -- train_loss: 1.3207 
2025-08-10 16:23:23,487 - INFO - Epoch: 1/500, Iter: 8/119 -- train_loss: 1.3197 
2025-08-10 16:23:23,839 - INFO - Epoch: 1/500, Iter: 9/119 -- train_loss: 1.3195 
2025-08-10 16:23:27,484 - INFO - Epoch: 1/500, Iter: 10/119 -- train_loss: 1.2773 
2025-08-10 16:23:27,838 - INFO - Epoch: 1/500, Iter: 11/119 -- train_loss: 1.3014 
2025-08-10 16:23:28,162 - INFO - Epoch: 1/500, Iter: 12/119 -- train_loss: 1.3029 
2025-08-10 16:23:28,491 - INFO - Epoch: 1/500, Iter: 13/119 -- train_loss: 1.3017 
2025-08-10 1

`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.


...

2025-08-11 18:25:19,652 - INFO - Epoch: 334/500, Iter: 2/119 -- train_loss: 1.1741 
2025-08-11 18:25:21,437 - INFO - Epoch: 334/500, Iter: 3/119 -- train_loss: 1.1658 
2025-08-11 18:25:25,929 - INFO - Epoch: 334/500, Iter: 4/119 -- train_loss: 1.1761 
2025-08-11 18:25:26,226 - INFO - Epoch: 334/500, Iter: 5/119 -- train_loss: 1.1378 
2025-08-11 18:25:26,509 - INFO - Epoch: 334/500, Iter: 6/119 -- train_loss: 1.1709 
2025-08-11 18:25:26,768 - INFO - Epoch: 334/500, Iter: 7/119 -- train_loss: 1.1701 
2025-08-11 18:25:27,043 - INFO - Epoch: 334/500, Iter: 8/119 -- train_loss: 1.1741 
2025-08-11 18:25:27,356 - INFO - Epoch: 334/500, Iter: 9/119 -- train_loss: 1.1707 
2025-08-11 18:25:27,734 - INFO - Epoch: 334/500, Iter: 10/119 -- train_loss: 1.1637 
2025-08-11 18:25:28,046 - INFO - Epoch: 334/500, Iter: 11/119 -- train_loss: 1.1741 
2025-08-11 18:25:32,327 - INFO - Epoch: 334/500, Iter: 12/119 -- train_loss: 1.1741 
2025-08-11 18:25:32,641 - INFO - Epoch: 334/500, Iter: 13/119 -- train_lo

[1/20]   5%|5          [00:00<?]

Engine run is terminating due to exception: 


2025-08-11 18:28:18,962 - ERROR - Exception: 
Traceback (most recent call last):
  File "c:\Users\anson\OneDrive\Documents\p158-ap\p158-env\Lib\site-packages\ignite\engine\engine.py", line 1032, in _run_once_on_dataset_as_gen
    self.state.batch = next(self._dataloader_iter)
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\anson\OneDrive\Documents\p158-ap\p158-env\Lib\site-packages\torch\utils\data\dataloader.py", line 733, in __next__
    data = self._next_data()
           ^^^^^^^^^^^^^^^^^
  File "c:\Users\anson\OneDrive\Documents\p158-ap\p158-env\Lib\site-packages\torch\utils\data\dataloader.py", line 1480, in _next_data
    raise StopIteration
StopIteration

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\anson\OneDrive\Documents\p158-ap\p158-env\Lib\site-packages\ignite\engine\engine.py", line 959, in _internal_run_as_gen
    epoch_time_taken += yield from self._run_once_on_dataset_as_gen

KeyboardInterrupt: 

In [ ]:
import os
from prostate158.inference3 import inference_pipeline

# Set paths for case ID 20
data_dir = "prostate158"  # Base directory from config
case_dir = os.path.join(data_dir, "train", "051")

# Input paths
t2_path = os.path.join(case_dir, "t2.nii.gz")
adc_path = os.path.join(case_dir, "adc.nii.gz")
dwi_path = os.path.join(case_dir, "dwi.nii.gz")

# Output path
os.makedirs("predictions", exist_ok=True)
output_path = os.path.join("predictions", "case_051_tumor_pred.nii.gz")

# Run inference
inference_pipeline(
    t2_path=t2_path,
    adc_path=adc_path,
    dwi_path=dwi_path,
    output_path=output_path,
    config_path="tumor.yaml",
    checkpoint_path="models/tumor.pt",
)

Finish the training with final evaluation of the best model. To allow visualization of all outputs, add OutputStore handler first. Otherwise only output form the last epoch will be accessible. 

In [ ]:
eos_handler = ignite.handlers.EpochOutputStore()
eos_handler.attach(trainer.evaluator, "output")

In [ ]:
trainer.evaluate(checkpoint="models/<model-name>.pt")

Generate a markdown document with segmentation results

In [ ]:
report_generator = ReportGenerator(config.run_id, config.out_dir, config.log_dir)
report_generator.generate_report()

Have a look at some outputs

In [ ]:
output = trainer.evaluator.state.output
keys = ["image", "label", "pred"]
outputs = {k: [o[0][k].detach().cpu().squeeze() for o in output] for k in keys}

In [ ]:
ListViewer(
    [o.transpose(0, 2).flip(-2) for o in outputs["image"][0:3]]
    + [o.argmax(0).transpose(0, 2).flip(-2).float() for o in outputs["label"][0:3]]
    + [o.argmax(0).transpose(0, 2).flip(-2).float() for o in outputs["pred"][0:3]]
).show()